# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LORA in order to do an efficient fine tuning
* Model: GPT 2
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
#!pip install transformers datasets

In [2]:
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset('dair-ai/emotion')

Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 31819.26 examples/s]


In [4]:
dataset['train'][2]

{'text': 'im grabbing a minute to post i feel greedy wrong', 'label': 3}

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
dataset = dataset.rename_column("label", "labels")

In [7]:
num_classes=len(set(dataset['test']['labels']))

In [8]:
from transformers import (GPT2Tokenizer, GPT2Model, Trainer, TrainingArguments, DataCollatorWithPadding, GPT2ForSequenceClassification, AutoTokenizer, GPT2Config)

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [9]:
# #tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# # default to left padding
# tokenizer.padding_side = "left"
# # Define PAD Token = EOS Token = 50256


In [10]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, return_tensors="pt")

# preprocess data set
tokenized_dataset = {}
for split in ['train', 'test']:
    tokenized_dataset[split] = dataset[split].map(preprocess_function, batched=True)
    tokenized_dataset[split] = tokenized_dataset[split].remove_columns(dataset[split].column_names)


tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 156kB/s]
config.json: 100%|██████████| 665/665 [00:00<00:00, 3.62MB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 15.4MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 11.4MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 32.9MB/s]
Map: 100%|██████████| 2000/2000 [00:01<00:00, 1622.72 examples/s]


In [11]:
# tokenizer = AutoTokenizer.from_pretrained('gpt2')

config = GPT2Config.from_pretrained(
    'gpt2', 
    num_labels=num_classes,
)
#     id2label={0:"sadness", 1:"joy", 2:"love", 3:"anger", 4:"fear", 5:"surprise"},
#     label2id={"sadness":0, "joy":1, "love":2, "anger":3, "fear":4, "surprise":5}),
config.pad_token_id = 50256

model = GPT2ForSequenceClassification(config)

In [12]:
# evaluate base model
#compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# eval loop
trainer = Trainer(
    model=model,
    args=TrainingArguments(
            output_dir="./data/emotion",
            per_device_eval_batch_size=16,
    ),
    train_dataset=None,
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
    )

In [ ]:
evaluation_results = trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [13]:
from peft import LoraConfig

In [14]:
config = LoraConfig()

In [15]:
# config_model = GPT2Config.from_pretrained('gpt2', num_labels=num_classes)
# config_model.pad_token_id = 50256

# model = GPT2ForSequenceClassification(config_model)

In [16]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [17]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,739,328 || trainable%: 0.23642263007862283


In [18]:
# eval loop
trainer_lora = Trainer(
    model=lora_model,
    args=TrainingArguments(
            output_dir="./data/emotion",
            learning_rate=2e-5,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=1,
            weight_decay=0.01,
            load_best_model_at_end=True,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            ),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
    )
trainer_lora.train()

IndexError: Invalid key: 15286 is out of bounds for size 0

In [ ]:
#save the model
lora_model.save_pretrained('gpt2-lora')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
lora_model = GPT2ForSequenceClassification.from_pretrained("gpt-lora")

In [ ]:
trainer_lora.evaluate()